In [1]:
import zarr
import numpy as np
#Loading Required Libraries 
import anndata
import scipy as sp
import geopandas
#import squidpy as sq
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pandas as pd
from libpysal.weights import Queen, Rook, KNN
from esda.moran import Moran
from splot.esda import plot_moran
import pyseq
from pyseq import image_analysis as ia
import dask
#dask.config.set({'distributed.worker.multiprocessing-method': 'spawn'})
import warnings
warnings.filterwarnings('ignore')
from dask.distributed import Client
import xarray as xr
import dask
import skimage
import numba
%run /gpfs/commons/home/jsingh/util_pyseq.ipynb

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
im = ia.get_HiSeqImages(image_path = 'zarrs/m387ntga2.zarr')

ImageAnalysis::Opened m387ntga2 


In [3]:
one_z_plane = im.im.sel(obj_step = 8263, channel = 610, cycle=1)
labels = skimage.io.imread('/gpfs/commons/groups/nygcfaculty/PySeq/20210428_mouse_genotype_2/segmented_sections/m387ntga2_labels.tiff')

In [4]:
#format = one_z_plane_obj_step_channel_cycle
array_object_list = []
name_object_list = []
for i in im.im['channel'].values:
    for j in im.im['cycle'].values:
        #for k #in im.im['obj_step'].values[1]:
            k = 8028
            name = "one_x_plane_"+str(i)+"_"+str(j)+"_"+str(k)
            name_object_list.append(name)
            nme = im.im.sel(obj_step = k, cycle=j, channel = i)
            array_object_list.append(nme)
        

In [5]:
pl = array_object_list[0].values

In [6]:
pl

array([[116, 113, 113, ..., 121, 115, 117],
       [112, 114, 110, ..., 116, 115, 118],
       [110, 113, 112, ..., 118, 115, 116],
       ...,
       [109, 114, 113, ..., 111, 113, 111],
       [113, 111, 113, ..., 112, 115, 111],
       [111, 112, 112, ..., 115, 116, 111]], dtype=int16)

In [10]:
from numba import njit, prange
from time import time


In [25]:
@njit(parallel=True, nopython = True)
def get_pixels(lab_max):
    result_list = []
    for i in range(lab_max):
        result_list.append(pl[labels == i+1].mean())
    return result_list

In [26]:
pl[labels == i+2]

array([369, 385, 376, 353, 358, 394, 386, 370, 358, 424, 413, 393, 382,
       447, 432, 419, 402, 389, 468, 464, 447, 438], dtype=int16)

In [27]:
get_pixels(5)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(readonly array(int16, 2d, C), array(bool, 2d, C))
 
There are 22 candidate implementations:
      - Of which 20 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(readonly array(int16, 2d, C), array(bool, 2d, C))':
       No match.
      - Of which 2 did not match due to:
      Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 166.
        With argument(s): '(readonly array(int16, 2d, C), array(bool, 2d, C))':
       Rejected as the implementation raised a specific error:
         NumbaTypeError: unsupported array index type array(bool, 2d, C) in [array(bool, 2d, C)]
  raised from /gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/numba/core/typing/arraydecl.py:72

During: typing of intri

In [11]:
def get_cluster(queue_name = 'pe2', log_dir=None):
    """ Make dask cluster w/ workers = 2 cores, 32 G mem, and 1 hr wall time.

        return cluster, client
    """

    if log_dir is None:
        log_dir = join(getcwd(),'dask_logs')
        makedirs(log_dir, exist_ok=True)

    cluster = SLURMCluster(
                queue = queue_name, 
                cores = 6 ,
                memory = '36G',
                walltime='1:00:00',
                log_directory=log_dir,
                extra=["--lifetime", "55m", "--lifetime-stagger", "4m"])
    client = Client(cluster)

    return cluster, client

cluster, client = get_cluster()

In [12]:
print(cluster.dashboard_link)

http://10.4.200.57:8787/status


In [13]:
#cluster.scale(48)
cluster.scale(cores = 15)  # Or specify cores or memory directly

In [14]:
persisted_plane = array_object_list[0].persist()

In [15]:
persisted_plane

<xarray.DataArray 'm387ntga2' (row: 10467, col: 10752)>
dask.array<getitem, shape=(10467, 10752), dtype=int16, chunksize=(10467, 1792), chunktype=numpy.ndarray>
Coordinates:
    channel   int64 558
    cycle     int64 1
    obj_step  int64 8028
Dimensions without coordinates: row, col
Attributes:
    first_group:  0
    machine:      None
    scale:        1
    overlap:      256
    fixed_bg:     1

In [16]:
def get_pixels_persisted(lab):
    m = persisted_plane.values[labels == lab+1].mean()
    return m


In [17]:
%%time
from joblib import Parallel, delayed
from joblib import parallel_backend
with parallel_backend('dask',scheduler_host=cluster.scheduler._address):
    results = Parallel(n_jobs=-2)(delayed(get_pixels_persisted)(lab) for lab in range(10))
    

CPU times: user 2.82 s, sys: 4.67 s, total: 7.49 s
Wall time: 8.98 s


In [37]:
file_path = 'Mean_Intensities/' + str(name_object_list[0]) + '.pkl'

In [38]:
file_path

'Mean_Intensities/one_x_plane_558_1_8028.pkl'

In [40]:
import pickle
with open(file_path, 'wb') as b:
    pickle.dump(results,b)

In [ ]:
#dask delayed version
@dask.delayed
def get_pixels(lab):
    m = one_z_plane.values[labels == lab+1].mean()
    return m

some_list = []
for i in range(500):
    some_list.append(get_pixels(i))
    

graph = dask.delayed()(some_list)
# graph.visualize()
computed_list = graph.compute()



In [27]:
client.restart()

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.4.200.45:41348/status,
Dashboard: http://10.4.200.45:41348/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.4.200.45:46810,Workers: 0
Dashboard: http://10.4.200.45:41348/status,Total threads: 0
Started: Just now,Total memory: 0 B


2022-06-17 18:05:31,559 - bokeh.core.property.validation - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/distributed/utils.py", line 767, in wrapper
    return func(*args, **kwargs)
  File "/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 355, in update
    self.root.title.text = title
AttributeError: 'str' object has no attribute 'text'
2022-06-17 18:05:31,570 - bokeh.application.handlers.function - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/distributed/utils.py", line 767, in wrapper
    return func(*args, **kwargs)
  File "/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 3915, in status_doc
    clus

In [41]:
client.close()

In [42]:
cluster.close()

2022-06-19 17:22:50,117 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
